In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/MURI Aug17 Thin Plate'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
b_value = np.array([0.0,0.25,0.5,1.0]).reshape(-1,1)


LR_tune, B_value = np.meshgrid(lr_tune,b_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
B_value = B_value.flatten('F').reshape(-1,1)


lrb_tune = np.hstack((LR_tune,B_value))

In [6]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
loss_thresh = 1

data = scipy.io.loadmat('../cylinder_nektar_wake.mat')
           
U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# # Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None].reshape(-1,1) # NT x 1
y = YY.flatten()[:,None].reshape(-1,1) # NT x 1
t = TT.flatten()[:,None].reshape(-1,1) # NT x 1

u_true = UU.flatten()[:,None].reshape(-1,1) # NT x 1
v_true = VV.flatten()[:,None].reshape(-1,1) # NT x 1
p_true = PP.flatten()[:,None].reshape(-1,1) # NT x 1

label = "3D_NS_stan"

loss_thresh = 10000

N_train = x.shape[0]
xyt = np.hstack((x,y,t))

lb_xyt = xyt[0]
ub_xyt = xyt[-1]

# xyt_test_tensor = torch.from_numpy(xyt).float().to(device)
np.random.seed(1234)
idx = np.random.choice(N_train, 5000, replace=False)
u_true_test = u_true[idx,:]
v_true_test = v_true[idx,:]
p_true_test = p_true[idx,:]

u_true_norm = np.linalg.norm(u_true,2)
v_true_norm = np.linalg.norm(v_true,2)
p_true_norm = np.linalg.norm(p_true,2)

# u_true_test = torch.from_numpy(u_true_test).float().to(device)
# v_true_test = torch.from_numpy(v_true_test).float().to(device)
# p_true_test = torch.from_numpy(p_true_test).float().to(device)

x_tensor = torch.from_numpy(x[idx,:]).float().to(device)
y_tensor = torch.from_numpy(y[idx,:]).float().to(device)
t_tensor = torch.from_numpy(t[idx,:]).float().to(device)

In [7]:
def trainingdata(N_T,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    idx = np.random.choice(N_train, N_T, replace=False)
    x_train = x[idx,:]
    y_train = y[idx,:]
    t_train = t[idx,:]
    u_train = u_true[idx,:]
    v_train = v_true[idx,:]
    
    return x_train,y_train,t_train,u_train,v_train

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,beta_init):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
        
        self.beta = Parameter(beta_init*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
        
        self.lambda1 = Parameter(torch.tensor(0.0))
        self.lambda1.requires_grad = True
        
        self.lambda2 = Parameter(torch.tensor(0.0))
        self.lambda2.requires_grad = True
    
            
    'foward pass'
    def forward(self,xyt):
        if torch.is_tensor(xyt) != True:         
            xyt = torch.from_numpy(xyt)                
        
        ubxyt = torch.from_numpy(ub_xyt).float().to(device)
        lbxyt = torch.from_numpy(lb_xyt).float().to(device)
    
                      
        #preprocessing input 
        xyt = (xyt - lbxyt)/(ubxyt - lbxyt)
        
        #convert to float
        a = xyt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            z1 = self.activation(z) 
            a = z1 + self.beta[:,i]*z*z1
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_uv(self,x_train,y_train,t_train,u_train,v_train):
        x1 = x_train.clone()
        x1.requires_grad = True
        
        y1 = y_train.clone()
        y1.requires_grad = True
        
        t1 = t_train.clone()
        t1.requires_grad = True
        
#         print(torch.cat((x1,y1,t1),dim=1).shape)
        psi_p = self.forward(torch.cat((x1,y1,t1),dim =1))
        
        psi = psi_p[:,0:1]
        
#         print(psi.shape)
        psi_x = autograd.grad(psi,x1,torch.ones([x1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        psi_y = autograd.grad(psi,y1,torch.ones([y1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
                
        loss_u = self.loss_function(psi_y,u_train)
        loss_v = self.loss_function(-1*psi_x,v_train)
                
        return loss_u + loss_v
    
    def loss_PDE(self, x_train,y_train,t_train,fg_hat):
        
        x1 = x_train.clone()
        x1.requires_grad = True
        
        y1 = y_train.clone()
        y1.requires_grad = True
        
        t1 = t_train.clone()
        t1.requires_grad = True
        
        psi_p = self.forward(torch.cat((x1,y1,t1),dim =1))
        psi = psi_p[:,0:1]
        p = psi_p[:,1:2]
        
        
        u = autograd.grad(psi,y1,torch.ones([y1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        v = -1*autograd.grad(psi,x1,torch.ones([x1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        #---------------------------------------------------------------------------------------------------------------
        u_t = autograd.grad(u,t1,torch.ones([t1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_x = autograd.grad(u,x1,torch.ones([x1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        u_xx = autograd.grad(u_x,x1,torch.ones([x1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
                
        u_y = autograd.grad(u,y1,torch.ones([y1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        u_yy = autograd.grad(u_y,y1,torch.ones([y1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        #---------------------------------------------------------------------------------
        
        v_t = autograd.grad(v,t1,torch.ones([t1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        v_x = autograd.grad(v,x1,torch.ones([x1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        v_xx = autograd.grad(v_x,x1,torch.ones([x1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
                
        v_y = autograd.grad(v,y1,torch.ones([y1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        v_yy = autograd.grad(v_y,y1,torch.ones([y1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        #------------------------------------------------------------------------------------
        p_x = autograd.grad(p,x1,torch.ones([x1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        p_y = autograd.grad(p,y1,torch.ones([y1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
            
            

        f = u_t + self.lambda1*(u*u_x + v*u_y) + p_x - self.lambda2*(u_xx + u_yy)
        g = v_t + self.lambda1*(u*v_x + v*v_y) + p_y - self.lambda2*(v_xx + v_yy)
        
        loss_f = self.loss_function(f,fg_hat)
        loss_g = self.loss_function(g,fg_hat)
                
        return loss_f + loss_g
    
    def loss(self,x_train,y_train,t_train,u_train,v_train,fg_hat):

        loss_uv = self.loss_uv(x_train,y_train,t_train,u_train,v_train)
        loss_fg = self.loss_PDE(x_train,y_train,t_train,fg_hat)
        loss_val = loss_uv + loss_fg
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
       
    'test neural network'
    def test(self):
        x1 = x_tensor.clone()
        x1.requires_grad = True
        
        y1 = y_tensor.clone()
        y1.requires_grad = True
        
        t1 = t_tensor.clone()
        t1.requires_grad = True
        
        psi_p_pred = self.forward(torch.cat((x1,y1,t1),dim =1))
        psi = psi_p_pred[:,0:1]
        p_pred = psi_p_pred[:,1:2]
        
        u_pred = autograd.grad(psi,y1,torch.ones([y1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        v_pred = -1*autograd.grad(psi,x1,torch.ones([x1.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
   
        u_pred = u_pred.cpu().detach().numpy()
        v_pred = v_pred.cpu().detach().numpy()
        p_pred = p_pred.cpu().detach().numpy()
    
        return u_pred,v_pred,p_pred

    def test_loss(self):
        u_pred,v_pred,p_pred = self.test()
        
        test_mse_u = np.mean(np.square(u_pred.reshape(-1,1) - u_true_test.reshape(-1,1)))
        test_re_u = np.linalg.norm(u_pred.reshape(-1,1) - u_true_test.reshape(-1,1),2)/u_true_norm
        
        test_mse_v = np.mean(np.square(v_pred.reshape(-1,1) - v_true_test.reshape(-1,1)))
        test_re_v = np.linalg.norm(v_pred.reshape(-1,1) - v_true_test.reshape(-1,1),2)/v_true_norm
        
        test_mse_p = np.mean(np.square(p_pred.reshape(-1,1) - p_true_test.reshape(-1,1)))
        test_re_p = np.linalg.norm(p_pred.reshape(-1,1) - p_true_test.reshape(-1,1),2)/p_true_norm
        
        return test_mse_u, test_re_u, test_mse_v, test_re_v, test_mse_p, test_re_p 

In [9]:
def train_step(x_train,y_train,t_train,u_train,v_train,fg_hat,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_train,y_train,t_train,u_train,v_train,fg_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [10]:
def data_update(loss_np,x_train,y_train,t_train):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    lambda1_val.append(PINN.lambda1.cpu().detach().numpy())
    lambda2_val.append(PINN.lambda2.cpu().detach().numpy())
    
    test_mse_u, test_re_u, test_mse_v, test_re_v, test_mse_p, test_re_p = PINN.test_loss()
    
    test_mse_u_loss.append(test_mse_u)
    test_re_u_loss.append(test_re_u)
    
    test_mse_v_loss.append(test_mse_v)
    test_re_v_loss.append(test_re_v)
    
    test_mse_p_loss.append(test_mse_p)
    test_re_p_loss.append(test_re_p)

In [11]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    x_train,y_train,t_train,u_train,v_train = trainingdata(N_T,(reps)*22)

    x_train = torch.from_numpy(x_train).float().to(device)
    y_train = torch.from_numpy(y_train).float().to(device)
    t_train = torch.from_numpy(t_train).float().to(device)
    u_train = torch.from_numpy(u_train).float().to(device)
    v_train = torch.from_numpy(v_train).float().to(device)
        
    fg_hat = torch.zeros(x_train.shape[0],1).to(device)
    
    for i in range(max_iter):
        train_step(x_train,y_train,t_train,u_train,v_train,fg_hat,i)

        loss_np = PINN.loss(x_train,y_train,t_train,u_train,v_train,fg_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np,x_train,y_train,t_train)
        print(i,"Loss %.4f"%train_loss[-1],"TestP: %.4f"%test_re_p_loss[-1], "L1 %.4f"%lambda1_val[-1],"L2 %.4f"%lambda2_val[-1])

    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [ ]:
nan_tune = []
for tune_reps in range(20):

    label = "Inv_NS_stan" + str(tune_reps)

    max_reps = 10
    max_iter = 100 #1000

    train_loss_full = []

    test_mse_u_full = []
    test_re_u_full = []
    test_mse_v_full = []
    test_re_v_full = []
    test_mse_p_full = []
    test_re_p_full = []


    beta_full = []
    lambda1_full = []
    lambda2_full = []

    elapsed_time= np.zeros((max_reps,1))

    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))

    beta_init = lrb_tune[tune_reps,1]

    for reps in range(max_reps):
        print(label)
        print(reps)
        train_loss = []

        test_mse_u_loss = []
        test_re_u_loss = []

        test_mse_v_loss = []
        test_re_v_loss = []

        test_mse_p_loss = []
        test_re_p_loss = []

        lambda1_val = []
        lambda2_val = []
        beta_val = []

        print(reps)

        torch.manual_seed(reps*36)
        N_T = 5000

        layers = np.array([3,50,50,50,50,50,50,50,50,50,2]) #9 hidden layers

        PINN = Sequentialmodel(layers,beta_init)

        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())


        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrb_tune[tune_reps,0], 
                                  max_iter = 20, 
                                  max_eval = 30, 
                                  tolerance_grad = 1e-8, 
                                  tolerance_change = 1e-8, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')



        nan_flag = train_model(max_iter,reps)


        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)

        test_mse_u_full.append(test_mse_u_loss)
        test_re_u_full.append(test_re_u_loss)
        test_mse_v_full.append(test_mse_v_loss)
        test_re_v_full.append(test_re_v_loss)
        test_mse_p_full.append(test_mse_p_loss)
        test_re_p_full.append(test_re_p_loss)

        #elapsed_time[reps] = time.time() - start_time
        beta_full.append(beta_val)
        lambda1_full.append(lambda1_val)
        lambda2_full.append(lambda2_val)

        #print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_u": test_mse_u_full,"test_re_u_loss": test_re_u_full,"test_mse_v": test_mse_v_full,"test_re_v_loss": test_re_v_full,"test_mse_p": test_mse_p_full,"test_re_p_loss": test_re_p_full,"Time": elapsed_time, "beta": beta_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold, "lambda1":lambda1_full,"lambda2":lambda2_full}
    savemat(label+'.mat', mdic)

Inv_NS_stan0
0
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=2, bias=True)
  )
)
0
0 Loss 0.1580 TestP: 0.1052 L1 0.0015 L2 0.0143
1 Loss 0.1491 TestP: 0.1137 L1 -0.0042 L2 0.0291
2 Loss 0.1300 TestP: 0.2773 L1 -0.0228 L2 0.0395
3 Loss 0.1246 TestP: 0.3243 L1 -0.0246 L2 0.0223
4 Loss 0.1205 TestP: 0.3511 L1 -0.0359 L2 0.0096
5 Loss 0.1164 TestP: 0.4051 L1 -0.0275 

36 Loss 0.0944 TestP: 0.1283 L1 -0.0356 L2 -0.0012
37 Loss 0.0939 TestP: 0.1333 L1 -0.0247 L2 -0.0008
38 Loss 0.0935 TestP: 0.1424 L1 -0.0167 L2 -0.0009
39 Loss 0.0929 TestP: 0.1829 L1 0.0385 L2 0.0003
40 Loss 0.0926 TestP: 0.2067 L1 0.0707 L2 0.0009
41 Loss 0.0921 TestP: 0.1985 L1 0.1102 L2 0.0013
42 Loss 0.0898 TestP: 0.2012 L1 0.1231 L2 -0.0002
43 Loss 0.0860 TestP: 0.2190 L1 0.2097 L2 0.0035
44 Loss 0.0821 TestP: 0.2206 L1 0.5537 L2 0.0151
45 Loss 0.0779 TestP: 0.1898 L1 0.7652 L2 0.0183
46 Loss 0.0571 TestP: 0.1592 L1 0.8700 L2 0.0147
47 Loss 0.0449 TestP: 0.2343 L1 0.7731 L2 0.0146
48 Loss 0.0370 TestP: 0.2447 L1 0.8275 L2 0.0117
49 Loss 0.0329 TestP: 0.1619 L1 0.8233 L2 0.0105
50 Loss 0.0290 TestP: 0.1012 L1 0.8497 L2 0.0107
51 Loss 0.0262 TestP: 0.0759 L1 0.8356 L2 0.0099
52 Loss 0.0236 TestP: 0.0756 L1 0.8323 L2 0.0091
53 Loss 0.0213 TestP: 0.0630 L1 0.8704 L2 0.0131
54 Loss 0.0200 TestP: 0.0766 L1 0.8619 L2 0.0123
55 Loss 0.0189 TestP: 0.0960 L1 0.8656 L2 0.0129
56 Loss 0.017

88 Loss 0.0890 TestP: 0.0552 L1 0.0350 L2 0.0001
89 Loss 0.0890 TestP: 0.0554 L1 0.0367 L2 0.0000
90 Loss 0.0889 TestP: 0.0576 L1 0.0339 L2 -0.0004
91 Loss 0.0888 TestP: 0.0660 L1 0.0333 L2 -0.0005
92 Loss 0.0886 TestP: 0.0563 L1 0.0296 L2 -0.0011
93 Loss 0.0885 TestP: 0.0573 L1 0.0371 L2 -0.0011
94 Loss 0.0882 TestP: 0.0673 L1 0.0493 L2 -0.0003
95 Loss 0.0879 TestP: 0.0554 L1 0.0698 L2 -0.0000
96 Loss 0.0872 TestP: 0.0558 L1 0.1131 L2 -0.0011
97 Loss 0.0860 TestP: 0.0628 L1 0.1646 L2 -0.0006
98 Loss 0.0835 TestP: 0.0573 L1 0.2288 L2 0.0042
99 Loss 0.0811 TestP: 0.0583 L1 0.3374 L2 0.0067
Training time: 521.08
Inv_NS_stan0
3
3
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, 

25 Loss 0.0953 TestP: 0.3943 L1 -0.0087 L2 0.0005
26 Loss 0.0953 TestP: 0.3857 L1 0.0067 L2 0.0008
27 Loss 0.0952 TestP: 0.3701 L1 0.0048 L2 0.0008
28 Loss 0.0952 TestP: 0.3913 L1 0.0046 L2 0.0008
29 Loss 0.0951 TestP: 0.3956 L1 -0.0041 L2 0.0008
30 Loss 0.0951 TestP: 0.4195 L1 -0.0231 L2 0.0006
31 Loss 0.0951 TestP: 0.4459 L1 -0.0248 L2 0.0009
32 Loss 0.0950 TestP: 0.4265 L1 -0.0212 L2 0.0008
33 Loss 0.0950 TestP: 0.4277 L1 -0.0178 L2 0.0007
34 Loss 0.0950 TestP: 0.4381 L1 -0.0122 L2 0.0008
35 Loss 0.0949 TestP: 0.4492 L1 -0.0047 L2 0.0010
36 Loss 0.0949 TestP: 0.4717 L1 -0.0015 L2 0.0014
37 Loss 0.0948 TestP: 0.4942 L1 -0.0131 L2 0.0010
38 Loss 0.0948 TestP: 0.5213 L1 0.0006 L2 0.0011
39 Loss 0.0947 TestP: 0.5256 L1 -0.0008 L2 0.0010
40 Loss 0.0947 TestP: 0.5425 L1 0.0077 L2 0.0011
41 Loss 0.0946 TestP: 0.5734 L1 0.0062 L2 0.0012
42 Loss 0.0946 TestP: 0.5845 L1 0.0095 L2 0.0012
43 Loss 0.0945 TestP: 0.6721 L1 0.0001 L2 0.0015
44 Loss 0.0945 TestP: 0.6563 L1 0.0131 L2 0.0014
45 Loss 0

77 Loss 0.0103 TestP: 1.3909 L1 0.9162 L2 0.0159
78 Loss 0.0100 TestP: 1.4103 L1 0.9266 L2 0.0161
79 Loss 0.0096 TestP: 1.4306 L1 0.9367 L2 0.0174
80 Loss 0.0093 TestP: 1.4301 L1 0.9383 L2 0.0170
81 Loss 0.0089 TestP: 1.4495 L1 0.9341 L2 0.0165
82 Loss 0.0086 TestP: 1.4806 L1 0.9399 L2 0.0159
83 Loss 0.0082 TestP: 1.5218 L1 0.9457 L2 0.0158
84 Loss 0.0079 TestP: 1.5602 L1 0.9434 L2 0.0162
85 Loss 0.0076 TestP: 1.5912 L1 0.9483 L2 0.0156
86 Loss 0.0072 TestP: 1.6224 L1 0.9547 L2 0.0151
87 Loss 0.0069 TestP: 1.6544 L1 0.9564 L2 0.0147
88 Loss 0.0065 TestP: 1.6880 L1 0.9578 L2 0.0152
89 Loss 0.0061 TestP: 1.7316 L1 0.9561 L2 0.0149
90 Loss 0.0059 TestP: 1.7779 L1 0.9642 L2 0.0144
91 Loss 0.0056 TestP: 1.8384 L1 0.9681 L2 0.0147
92 Loss 0.0053 TestP: 1.9129 L1 0.9664 L2 0.0143
93 Loss 0.0050 TestP: 1.9493 L1 0.9700 L2 0.0148
94 Loss 0.0049 TestP: 1.9615 L1 0.9689 L2 0.0149
95 Loss 0.0047 TestP: 1.9540 L1 0.9684 L2 0.0147
96 Loss 0.0045 TestP: 1.9595 L1 0.9741 L2 0.0151
97 Loss 0.0043 TestP